In [ ]:
!nvidia-smi -L

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
# Import necessary Libraries
import numpy as np
import os
import scipy.stats as st
import math
from sklearn import preprocessing
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, recall_score, f1_score #sklearn.metrics.classification
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from itertools import cycle
from math import exp
from math import log
import tensorflow as tf
import matplotlib.pyplot as plt

import keras
from keras import layers
from keras.models import Model, load_model
from keras.initializers import glorot_uniform
from keras.layers import Input, Dropout, Add, Dense 
from keras.layers import MaxPooling1D, Reshape, Activation
from keras.layers import BatchNormalization, Flatten, Conv1D
from google.colab import files

In [ ]:
def DataPreparation_DNN(data_input_file):
  data = np.load(data_input_file, allow_pickle=True)
  X = data['X']
  X = X[:, 0, :, :]
  Y = data['y']
  n_class = Y.shape[1]
  folds = data['folds']
  return X,Y,folds,n_class

In [ ]:
# Deep Learning Models

def build_model_FUSEDLSTMCNN_1(row,col,num_classes): 
  layers = [
      
      tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()),
      
      tf.keras.layers.LSTM(28,return_sequences=True,  input_shape=(row, col)),  # 28
      tf.keras.layers.LSTM(28,return_sequences=True),                           # 28
      tf.keras.layers.LSTM(28,return_sequences=True),                           # 28
      tf.keras.layers.Dropout(0.1),
      tf.keras.layers.Conv1D(filters=64,kernel_size=5,strides=2,activation='relu'),
      tf.keras.layers.MaxPool1D(pool_size=2,strides = 2),
      tf.keras.layers.Conv1D(filters=128,kernel_size=3,strides=2,activation='relu'),
      tf.keras.layers.GlobalAveragePooling1D(),        
      tf.keras.layers.BatchNormalization(),     
        
      tf.keras.layers.Dense(num_classes, activation = 'softmax')
       ]
      
  model = tf.keras.Sequential(layers)
  return model


def build_FUSEDLSTMCNN2_DNN(row,col,num_classes):

  tf.keras.initializers.GlorotNormal(234)
  input_layer = tf.keras.Input(shape=(row,col,1,))
  
  layer = tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten())(input_layer)
  layer = tf.keras.layers.LSTM(28,return_sequences=True)(layer)
  layer = tf.keras.layers.Dropout(0.1)(layer)
  layer = tf.keras.layers.LSTM(28,return_sequences=True)(layer)
  layer = tf.keras.layers.Dropout(0.1)(layer)
  layerconv1 = tf.keras.layers.Conv1D(filters=30,kernel_size=5,  strides=1) (layer)
  layerconv1 = tf.keras.layers.Activation('relu')(layerconv1)
  layerconv2 = tf.keras.layers.Conv1D(filters=40,kernel_size= 10,strides=1) (layer)
  layerconv2 = tf.keras.layers.Activation('relu')(layerconv2)
  layerconv3 = tf.keras.layers.Conv1D(filters=50,kernel_size= 15,strides=1) (layer)
  layerconv3 = tf.keras.layers.Activation('relu')(layerconv3)
  layerconv4 = tf.keras.layers.Conv1D(filters=60,kernel_size= 20,strides=1) (layer)
  layerconv4 = tf.keras.layers.Activation('relu')(layerconv4)
  max1= tf.reduce_max(layerconv1, 1)
  max2= tf.reduce_max(layerconv2, 1)
  max3= tf.reduce_max(layerconv3, 1)
  max4= tf.reduce_max(layerconv4, 1)
  concat_layer = tf.keras.layers.concatenate([max1,max2,max3,max4],1)
  layer= tf.keras.layers.Dense(num_classes, activation = 'softmax')   (concat_layer) 

  model = tf.keras.Model(inputs=input_layer, outputs=layer)
  return model


def identity_block(input, kernel_size, filters, stage, block):
  # Variables
  filters1, filters2, filters3 = filters
  conv_name_base = 'res' + str(stage) + block + '_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'
  # Create layers
  output = keras.layers.Conv1D(filters1, 1, kernel_initializer='he_normal', name=conv_name_base + '2a')(input)
  output = keras.layers.BatchNormalization(name=bn_name_base + '2a')(output)
  output = keras.layers.Activation('relu')(output)
  output = keras.layers.Conv1D(filters2, kernel_size, padding='same', kernel_initializer='he_normal', name=conv_name_base + '2b')(output)
  output = keras.layers.BatchNormalization(name=bn_name_base + '2b')(output)
  output = keras.layers.Activation('relu')(output)
  output = keras.layers.Conv1D(filters3, 1, kernel_initializer='he_normal', name=conv_name_base + '2c')(output)
  output = keras.layers.BatchNormalization(name=bn_name_base + '2c')(output)
  output = keras.layers.add([output, input])
  output = keras.layers.Activation('relu')(output)
  # Return a block
  return output


def conv_block(input, kernel_size, filters, stage, block, strides=2):
  # Variables
  filters1, filters2, filters3 = filters
  conv_name_base = 'res' + str(stage) + block + '_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'
  # Create block layers
  output = keras.layers.Conv1D(filters1, 1, strides=strides, kernel_initializer='he_normal', name=conv_name_base + '2a')(input)
  output = keras.layers.BatchNormalization(name=bn_name_base + '2a')(output)
  output = keras.layers.Activation('relu')(output)
  output = keras.layers.Conv1D(filters2, kernel_size, padding='same', kernel_initializer='he_normal', name=conv_name_base + '2b')(output)
  output = keras.layers.BatchNormalization(name=bn_name_base + '2b')(output)
  output = keras.layers.Activation('relu')(output)
  output = keras.layers.Conv1D(filters3, 1, kernel_initializer='he_normal', name=conv_name_base + '2c')(output)
  output = keras.layers.BatchNormalization(name=bn_name_base + '2c')(output)
  shortcut = keras.layers.Conv1D(filters3, 1, strides=strides, kernel_initializer='he_normal', name=conv_name_base + '1')(input)
  shortcut = keras.layers.BatchNormalization(name=bn_name_base + '1')(shortcut)
  output = keras.layers.add([output, shortcut])
  output = keras.layers.Activation('relu')(output)
  # Return a block
  return output



def build_model_ResNet(row, col, num_classes):
  input_shape = (row,col)
  # Create an input layer 
  input = keras.layers.Input(shape=input_shape)
  # Create output layers
  output = keras.layers.Conv1D(64, 7, strides=2, use_bias=False, name='conv1')(input)
  output = keras.layers.BatchNormalization(name='bn_conv1')(output)
  output = keras.layers.Activation('relu', name='conv1_relu')(output)
  output = keras.layers.MaxPooling1D(3, strides=2, padding='same', name='pool1')(output)
  output = conv_block(output, 3, [64, 64, 256], stage=2, block='a', strides=1)
  output = identity_block(output, 3, [64, 64, 256], stage=2, block='b')
  output = identity_block(output, 3, [64, 64, 256], stage=2, block='c')
  output = conv_block(output, 3, [128, 128, 512], stage=3, block='a')
  output = identity_block(output, 3, [128, 128, 512], stage=3, block='b')
  output = identity_block(output, 3, [128, 128, 512], stage=3, block='c')
  output = identity_block(output, 3, [128, 128, 512], stage=3, block='d')
  output = conv_block(output, 3, [256, 256, 1024], stage=4, block='a')
  output = identity_block(output, 3, [256, 256, 1024], stage=4, block='b')
  output = identity_block(output, 3, [256, 256, 1024], stage=4, block='c')
  output = identity_block(output, 3, [256, 256, 1024], stage=4, block='d')
  output = identity_block(output, 3, [256, 256, 1024], stage=4, block='e')
  output = identity_block(output, 3, [256, 256, 1024], stage=4, block='f')
  output = conv_block(output, 3, [512, 512, 2048], stage=5, block='a')
  output = identity_block(output, 3, [512, 512, 2048], stage=5, block='b')
  output = identity_block(output, 3, [512, 512, 2048], stage=5, block='c')
  output = keras.layers.GlobalAveragePooling1D(name='pool5')(output)
  output = keras.layers.Dense(num_classes, activation='softmax', name='fc1000')(output)
  # Create a model from input layer and output layers
  model = keras.models.Model(inputs=input, outputs=output)
  ## Print model
  # print(model.summary(), '\n')
  return model



In [ ]:
def Train_FUSEDLSTMCNN_1(X, y, X_train, X_test, y_train, y_test, n_class):
  _,img_rows, img_cols = X.shape
  X=X.reshape(X.shape[0],img_rows,img_cols,1)
  _,img_rows, img_cols,_ = X.shape

  model=build_model_FUSEDLSTMCNN_1(img_rows,img_cols,n_class)
  
  model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='ADAM')
  model.fit(X_train, y_train, batch_size=192, epochs=250,verbose=2)
  y_pred = model.predict(X_test)
  # if (i==2):
  confidence_factor_list = []
  for i in range(0, y_pred.shape[0]):
    total_fuzzy_sum = sum(y_pred[i])
    confidence_factor_activity = y_pred[i]/total_fuzzy_sum
    confidence_factor_list.append(confidence_factor_activity)
  confidence_factor_array = np.asarray(confidence_factor_list)

  return confidence_factor_array


def Train_FUSEDLSTMCNN_2(X, y, X_train, X_test, y_train, y_test, n_class):
  _, img_rows, img_cols = X.shape
  X=X.reshape(X.shape[0],img_rows,img_cols,1)
  _,img_rows, img_cols,_ = X.shape
    
  model=build_FUSEDLSTMCNN2_DNN(img_rows, img_cols, n_class)
  optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
  model.compile(loss="categorical_crossentropy", optimizer=optimizer,  metrics=['accuracy'])
  model.fit(X_train, y_train, batch_size=batch_size,epochs=200,verbose=2)
  y_pred = model.predict(X_test)
  confidence_factor_list = []
  for i in range(0, y_pred.shape[0]):
    total_fuzzy_sum = sum(y_pred[i])
    confidence_factor_activity = y_pred[i]/total_fuzzy_sum
    confidence_factor_list.append(confidence_factor_activity)
  confidence_factor_array = np.asarray(confidence_factor_list)

  return confidence_factor_array


def Train(X_Train, Y_Train, X_Test, Y_Test, clf):
  confidence_factor_list = []
  clf.fit(X_Train, Y_Train)
  f1 = clf.predict_proba(X_Test)

  for i in range(0, f1.shape[0]):
    total_fuzzy_sum = sum(f1[i])
    confidence_factor_activity = f1[i]/total_fuzzy_sum
    # print(confidence_factor_activity)
    confidence_factor_list.append(confidence_factor_activity)
  confidence_factor_array = np.asarray(confidence_factor_list) #, dtype=np.float32)
  # print("Confidence_factor_array shape: ", confidence_factor_array.shape)

  return confidence_factor_array, Y_Test


def Train_ResNet(X, y, X_train, X_test, y_train, y_test, n_class):
  _,img_rows, img_cols = X.shape
  X=X.reshape(X.shape[0],img_rows,img_cols,1)
  _,img_rows, img_cols,_ = X.shape

  model=build_model_ResNet(img_rows,img_cols,n_class)
  
  model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='ADAM')
  model.fit(X_train, y_train, batch_size=192, epochs=150,verbose=2)
  y_pred = model.predict(X_test)
  confidence_factor_list = []
  for i in range(0, y_pred.shape[0]):
    total_fuzzy_sum = sum(y_pred[i])
    confidence_factor_activity = y_pred[i]/total_fuzzy_sum
    confidence_factor_list.append(confidence_factor_activity)
  confidence_factor_array = np.asarray(confidence_factor_list)

  return confidence_factor_array


In [ ]:
# Functions
from math import exp
from math import log

def mitcherlich(time, alpha, beta, rate):
    """
    time : time
    alpha : upper asymptote
    beta : growth range
    rate : growth rate
    """
    result = alpha - beta * rate ** time
    return result


def blumberg(time, alpha, slope, w0=1):
    """
    time : time
    alpha : upper asymptote
    w0 : a reference value at time = time_0
    slope : slope of growth
    """
    result = (alpha * (time) ** slope) / (w0 + (time) ** slope)
    return result
    # result -- 1 (later -1 thing do)
    # result -- 0 (later -1 thing)


def weibull(time, alpha, beta, rate, slope):
    """
    time : time
    alpha : upper asymptote
    beta : growth displacement
    rate : growth rate
    slope : slope of growth
    """
    result = alpha - beta * exp(-rate * time ** slope)
    return result


In [ ]:
def GenerateFuzzyRank_Mitcherlich(confidence_factor_array):  
  fuzzy_rank_list = []
  for item in confidence_factor_array:
    arr=[]
    for confidence_factor in item:
      fuzzy_rank = mitcherlich(confidence_factor,2,1,2)
      arr.append(fuzzy_rank)
    fuzzy_rank_list.append(arr)

  return np.asarray(fuzzy_rank_list)

def GenerateFuzzyRank_Blumberg(confidence_factor_array):  
  fuzzy_rank_list = []
  for item in confidence_factor_array:
    arr=[]
    for confidence_factor in item:
      fuzzy_rank = 1 - blumberg(confidence_factor, 1, 0.0001, w0=0.5)
      arr.append(fuzzy_rank)
    fuzzy_rank_list.append(arr)

  return np.asarray(fuzzy_rank_list)

def GenerateFuzzyRank_Weibull(confidence_factor_array):  
  fuzzy_rank_list = []
  for item in confidence_factor_array:
    arr=[]
    for confidence_factor in item:
      fuzzy_rank = -weibull(confidence_factor, 0, 0.5, 2, 2) # We negate this 
      arr.append(fuzzy_rank)
    fuzzy_rank_list.append(arr)

  return np.asarray(fuzzy_rank_list)


In [ ]:
def getRFS(fuzzy_rank_array_1, fuzzy_rank_array_2, fuzzy_rank_array_3):
    RFS = []
    for j in range(len(fuzzy_rank_array_1)):
        arr = []
        temp_fuzzy_rank_array_1 = fuzzy_rank_array_1[j]
        temp_fuzzy_rank_array_2 = fuzzy_rank_array_2[j]
        temp_fuzzy_rank_array_3 = fuzzy_rank_array_3[j]
        for i in range(len(temp_fuzzy_rank_array_1)):
            arr.append(min(temp_fuzzy_rank_array_1[i], temp_fuzzy_rank_array_2[i], temp_fuzzy_rank_array_3[i]))
        RFS.append(arr)
    return np.asarray(RFS)

def getGR(RFS_1, RFS_2, RFS_3):
    GR = []
    for j in range(len(RFS_1)):
        arr = []
        temp_RFS_1 = RFS_1[j]
        temp_RFS_2 = RFS_2[j]
        temp_RFS_3 = RFS_3[j]
        for i in range(len(temp_RFS_1)):
            arr.append(temp_RFS_1[i]+temp_RFS_2[i]+temp_RFS_3[i])
        GR.append(arr)
    return np.asarray(GR)

def getGC(CFS_1, CFS_2, CFS_3):
    GC = []
    for j in range(len(CFS_1)):
        arr = []
        temp_CFS_1 = CFS_1[j]
        temp_CFS_2 = CFS_2[j]
        temp_CFS_3 = CFS_3[j]    
        for i in range(len(temp_CFS_1)):
            temp = (1-temp_CFS_1[i]) + (1-temp_CFS_2[i]) + (1-temp_CFS_3[i])
            arr.append(temp)
        GC.append(arr)
    return np.asarray(GC)

def getRF(CFS_1, CFS_2, CFS_3, S):
    RF = []
    for j in range(len(CFS_1)):  

        temp_CFS_1 = CFS_1[j]
        temp_CFS_2 = CFS_2[j]
        temp_CFS_3 = CFS_3[j]

        Store_1 = []
        Rank_1 = [0]*len(temp_CFS_1)
        for i in range(len(temp_CFS_1)):
            Store_1.append([i, temp_CFS_1[i]])
        Store_1 = sorted(Store_1, reverse=True, key=lambda x:x[1])
        for i in range(len(Rank_1)):
            for j in range(len(Store_1)):
                if i == Store_1[j][0]:
                    Rank_1[i] = j+1
                    break
    
        Store_2 = []
        Rank_2 = [0]*len(temp_CFS_2)
        for i in range(len(temp_CFS_2)):
            Store_2.append([i, temp_CFS_2[i]])
        Store_2 = sorted(Store_2, reverse=True, key=lambda x:x[1])
        for i in range(len(Rank_2)):
            for j in range(len(Store_2)):
                if i == Store_2[j][0]:
                    Rank_2[i] = j+1
                    break

        Store_3 = []
        Rank_3 = [0]*len(temp_CFS_3)
        for i in range(len(temp_CFS_3)):
            Store_3.append([i, temp_CFS_3[i]])
        Store_3 = sorted(Store_3, reverse=True, key=lambda x:x[1])
        for i in range(len(Rank_3)):
            for j in range(len(Store_3)):
                if i == Store_3[j][0]:
                    Rank_3[i] = j+1
                    break

        arr = []
        for i in range(len(temp_CFS_1)):
            temp = 1/((1 - ((Rank_1[i])/(S+1))) + (1 - ((Rank_2[i])/(S+1))) + (1 - ((Rank_3[i])/(S+1))))
            arr.append(temp)

        RF.append(arr)

    return np.asarray(RF)


def PredictClassFinal(GR, GC, RF):
    Y_Pred = []
    Y_Pred_Score = []

    for j in range(len(GR)): 
        temp_dict = dict()
        temp_GR = GR[j]
        temp_GC = GC[j]
        temp_RF = RF[j]   
        for i in range(len(temp_GR)):
            temp_dict[i] = temp_GR[i]*temp_GC[i]*temp_RF[i]
        ans = min(temp_dict, key=temp_dict.get)
        Y_Pred.append(ans)

        temp_y_pred_score = []
        for i in range(len(temp_dict)):
            score = 1/temp_dict[i]
            temp_y_pred_score.append(score)

        x_array = np.array(temp_y_pred_score)
        normalized_arr = preprocessing.normalize([x_array])
        Y_Pred_Score.append(np.array(normalized_arr[0])) 
    return np.asarray(Y_Pred), np.asarray(Y_Pred_Score)


In [ ]:
# Code for testing -- Uncomment and Test!!!!

if __name__ == "__main__":  
  X,y,folds,n_class = DataPreparation_DNN('/content/drive/MyDrive/SNOW/USCHAD.npz')

  avg_acc_list = []
  avg_recall_list = []
  avg_f1_list = []

  for i in range(0, len(folds)):
    train_idx = folds[i][0]
    test_idx = folds[i][1]
    X_train = X[train_idx]
    X_test = X[test_idx]
    y_train = y[train_idx]
    y_test = y[test_idx]

    # For classifier 1
    confidence_factor_array_1 = Train_FUSEDLSTMCNN_1(X, y, X_train, X_test, y_train, y_test, n_class)
    fuzzy_rank_array_1 = GenerateFuzzyRank_Mitcherlich(confidence_factor_array_1)
    fuzzy_rank_array_2 = GenerateFuzzyRank_Blumberg(confidence_factor_array_1)
    fuzzy_rank_array_3 = GenerateFuzzyRank_Weibull(confidence_factor_array_1)

    RFS_1 = getRFS(fuzzy_rank_array_1, fuzzy_rank_array_2, fuzzy_rank_array_3)

    
    # For classifier 2
    confidence_factor_array_2 = Train_FUSEDLSTMCNN_2(X, y, X_train, X_test, y_train, y_test, n_class)
    fuzzy_rank_array_4 = GenerateFuzzyRank_Mitcherlich(confidence_factor_array_2)
    fuzzy_rank_array_5 = GenerateFuzzyRank_Blumberg(confidence_factor_array_2)
    fuzzy_rank_array_6 = GenerateFuzzyRank_Weibull(confidence_factor_array_2)

    RFS_2 = getRFS(fuzzy_rank_array_4, fuzzy_rank_array_5, fuzzy_rank_array_6)


    
    # For classifier 3
    confidence_factor_array_3 = Train_ResNet(X, y, X_train, X_test, y_train, y_test, n_class)
    fuzzy_rank_array_7 = GenerateFuzzyRank_Mitcherlich(confidence_factor_array_3)
    fuzzy_rank_array_8 = GenerateFuzzyRank_Blumberg(confidence_factor_array_3)
    fuzzy_rank_array_9 = GenerateFuzzyRank_Weibull(confidence_factor_array_3)

    RFS_3 = getRFS(fuzzy_rank_array_7, fuzzy_rank_array_8, fuzzy_rank_array_9)


    GR = getGR(RFS_1, RFS_2, RFS_3)
    GC = getGC(confidence_factor_array_1, confidence_factor_array_2, confidence_factor_array_3)
    RF = getRF(confidence_factor_array_1, confidence_factor_array_2, confidence_factor_array_3, n_class)

    y_pred, y_score = PredictClassFinal(GR, GC, RF)

    # Correct format of y_test (becomes 1D)
    y_test = np.argmax(y_test, axis=1)

    
    acc_fold = accuracy_score(y_test, y_pred)
    avg_acc_list.append(acc_fold)
    recall_fold = recall_score(y_test, y_pred, average='macro')
    avg_recall_list.append(recall_fold)
    f1_fold  = f1_score(y_test, y_pred, average='macro')
    avg_f1_list.append(f1_fold)

    print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at Fold[{}]'.format(acc_fold, recall_fold, f1_fold ,i+1))
    print('________________________________________________________________')

  avg_acc = np.asarray(avg_acc_list)
  avg_recall = np.asarray(avg_recall_list)
  avg_f1 = np.asarray(avg_f1_list)
  print("\n")
  print('Overall Accuracy[{:.4f}] Overall Recall[{:.4f}] Overall F1[{:.4f}]'.format(np.mean(avg_acc), np.mean(avg_recall), np.mean(avg_f1)))

